## 决策树

### 引入

1. 微软小冰猜人
2. APP推荐
![JZWafU.png](https://s1.ax1x.com/2020/04/17/JZWafU.png)

### 如何决策拆分点呢？先来了解☝️熵

熵，是一个统计物理与信息论术语，表示的是某一系统“内在的混乱程度”。

我们可以从如下的示例中，直观的感受下熵的大小区别。
![JwImn0.png](https://s1.ax1x.com/2020/04/23/JwImn0.png)

在上图中，可以直观的感受到：
- 左边的箱子中只有红球，那么只有可能有一种结果，所以熵是最低的；
- 其次是中间，最高是最右侧的箱子。

那么具体到数值，应该怎样计算呢？

我们继续以上面三个箱子为例，假设我们随机的从各个箱子里面有放回的取样，如果取出来的结果与箱子中的排列完全一致，那么就判定为成功，否则为失败。那么请问，那个箱子的“成功率“最高呢？

答案显然是左边的箱子，我们利用概率计算验证一下：

- 左边：取红球的概率为1，取四次为独立事件，所以，完全一致的概率为 $1\times 1 \times 1 \times 1 = 1$；
- 中间：取红球的概率为0.75，取蓝球的概率为0.25，所以，完全一致的概率为 $0.75 \times 0.75 \times 0.75 \times 0.25 \approx  0.105$;
- 右边：红球蓝球的概率均为0.5，所以，完全一致的概率为 $0.5 \times 0.5 \times 0.5 \times 0.5 =  0.0625$



上述只是四个样本的情况，如果样本数量非常大，那么各个概率相乘得到结果必定会是一个极小的小数，为了避免这种情况，我们希望可以用和（sum）代替积（products），刚好可以使用log函数实现这个需求。

> 注意⚠️这里使用log计算时，选择的是以2为底的对数计算。

那么，可以计算得到左中右三个箱子的值分别为：0,`np.log2(0.75) * 3 + np.log2(0.25) = -3.245`, -4。

为了让结果能与混乱程度成正比，所以，这里取一个负数，然后再除以4，也就得到了三个系统的熵：0，0.81，1

所以，假如有5个红球，3个蓝球的话，那么这个系统的熵为：
$$ Entropy = -\frac{5}{8} \log_{2}\frac{5}{8} -\frac{3}{8} \log_{2}\frac{3}{8} = 0.9544$$

进而可以推广到多类别的熵计算：
$$entropy=−p_{1}\log_{2}(p_{1})−p_{2}\log_{2}(p_{2})−...−p_{n}\log_{2}(p_{n})=-\sum_{i=1}^{n}p_{i}\log_{2}(p_{i}
)$$

接下来，我们就可以依据熵增（information gain）来进行决策树的分裂了。

练习：如下图所示，哪一个的信息熵增最高呢？

![tqgNy8.png](https://s1.ax1x.com/2020/06/11/tqgNy8.png)

### 信息增益（information gain）

可以很明显得想到，3的信息增益是最高的，因为它是把红和蓝两类分的最清楚的一个，也就是说“我们对处理后的数据了解得比原始数据更多”，这就是信息增益。

如何计算信息增益呢？其实就是计算前后两个系统的熵差：

$$InformationGain=Entropy(Parent)−[\frac{m}{m+n}Entropy(Child_{1})+\frac{n}{m+n}Entropy(Child_{2})]$$

其中m为子节点1（Child_1）中的样本数量，n为子节点2的样本数量。

根据如上公式，可以计算出上述三种分类方法的信息增益分别为：0，0.28与1. 所以，作为子节点分裂来说，显然应该选择第三类分裂方法。

### 回顾app推荐

从前面的学习，我们可以得知，想要获取较好的模型性能，就需要决策树在每次分裂时都可以获得最大的信息增益，接下来，我们继续以app推荐为例，去探索如何选择分裂节点。

- 首先，我们来看下原始（根结点）系统的熵：
    $$ Entropy = -\frac{3}{6} \log_{2}\frac{3}{6} -\frac{2}{6} \log_{2}\frac{2}{6} -\frac{1}{6} \log_{2}\frac{1}{6}= 1.46$$

- 接下来，我们来看下，如果用`Gender`作为分裂点，去拆分后，会得到两个子节点，即：
    - F：两个whatsapp，一个pokeman go；熵为：0.92
    - M：一个snapchat，两个pokeman go；熵为：0.92
    
    所以，整个子系统的熵为0.92，也就是说信息增益为1.46-0.92=0.54.
 
- 我们再来看下，如果按`Occupation`作为分裂点，去拆分后，得到两个子节点，即：
    - Study：三个pokeman go；熵为：0
    - Work：两个Whatsapp，一个snapchat；熵为：0.92
    
    所以，整个子系统的熵为0.92/2 = 0.46，也就是说信息增益为1.

综上，我们比较出了按照`Occupation`作为分裂点是获得最高信息增益的情况。也就是说，先按照`Occupation`分裂，再按照`Gender`分裂的系统可以获得最佳的性能。

对于数值型变量来说，也是按照穷举的方式，比较出按照哪一个节点做分裂可以得到最高的信息增益，然后逐个往下分裂。如，最佳分裂点为数值5，那么就可以把数据分为<5和>=5两类。